## Image Captioner
We explore how to build a captioning model. It will rely on the CLIP model to perform inference. We will then use `gradio` to deploy it as a web app.

## Setting up Environment

In [1]:
# Checking CUDA version.
!nvcc --version

'nvcc' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
# !conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.8
# !pip install ftfy regex tqdm
# !pip install git+https://github.com/openai/CLIP.git
# !pip install gradio==4.7.1
# !pip install typing_extensions==4.7.1 --upgrade

In [2]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to c:\users\user\appdata\local\temp\pip-req-build-h96bd_m5
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 53.4/53.4 KB 345.2 kB/s eta 0:00:00
     ------------------------------------ 269.6/269.6 KB 922.8 kB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369620 sha256=5e545bb0b3b9a66efa528b39288fc5ed76a445609e4c234bcc36aee2a3922288
  Stored in directory: C:\Users\USER\AppData\Local\Temp\pip-ephem-wheel-cache-mlm9n9_g\wheels\da\2b\4c\d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip
  Attempting uninstall: wcwidth
    Found existing installation: wcwidth 0.2.6
    Uninstalling wcwidth-0.2.6:
      Successfully uninstalled wcwidth-0.2.6


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\USER\AppData\Local\Temp\pip-req-build-h96bd_m5'
You should consider upgrading via the 'F:\Jupyter\DeepLearning\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [7]:
!pip install typing_extensions==4.7.1 --upgrade

You should consider upgrading via the 'F:\Jupyter\DeepLearning\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
import clip
import torch
from PIL import Image
import gradio as gr

In [ ]:
image = Image.open("data/dog.jpg")

In [ ]:
# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

# Download the dataset
cifar100 = CIFAR100(root=os.path.expanduser("~/.cache"), download=True, train=False)

In [ ]:
image_input = preprocess(image).unsqueeze(0).to(device)
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in cifar100.classes]).to(device)

In [ ]:
# Calculate features
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

# Pick the top 5 most similar labels for the image
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
values, indices = similarity[0].topk(5)

# Print the result
print("\nTop predictions:\n")
for value, index in zip(values, indices):
    print(f"{cifar100.classes[index]:>16s}: {100 * value.item():.2f}%")


Top predictions:

             fox: 18.66%
    sweet_pepper: 10.32%
            girl: 7.12%
             boy: 6.09%
            wolf: 5.54%
